In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [2]:
house_df=pd.read_csv('house_price_train.csv ')
#test_df=pd.read_csv('house_price_test.csv ')

In [3]:

#house_df=pd.concat([train_df,test_df],axis=0)
house_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:



y=house_df['SalePrice']

house_df.drop(columns='SalePrice',inplace=True)




In [5]:
#house_df=house_df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [6]:
#missing value cols

missing_df=pd.DataFrame({'count':house_df.isnull().sum(),
                        'percentage':house_df.isnull().sum()/len(house_df)*100})
missing_df.sort_values(by='percentage',ascending=False,inplace=True)
missing_cols=list(missing_df[missing_df['percentage']>40].index)
missing_cols

['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']

In [7]:
#uni value cols
uni_cols=[]
for col in house_df.columns:
    if len(house_df[col].value_counts())==1 or len(house_df[col].value_counts())==len(house_df):
        uni_cols.append(col)
uni_cols

['Id']

In [8]:
house_df.drop(columns=list(set(missing_cols+uni_cols)),inplace=True)

In [9]:
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV

In [10]:
x_train,x_test,y_train,y_test=train_test_split(house_df,y,test_size=.20,random_state=42)

In [11]:
num_cols=list(house_df.select_dtypes(include=[np.number]).columns)
cat_cols=list(house_df.select_dtypes(include=[np.object]).columns)

In [12]:
for col in num_cols:
    x_train[col]=x_train[col].fillna(x_train[col].mean())
    x_test[col]=x_test[col].fillna(x_train[col].mean())

In [13]:
for col in cat_cols:
    x_train[col]=x_train[col].fillna(x_train[col].mode()[0])
    x_test[col]=x_test[col].fillna(x_train[col].mode()[0])
  

In [14]:
from sklearn.preprocessing import MinMaxScaler
scale=MinMaxScaler()
for col in num_cols:
    x_train[col]=scale.fit_transform(np.array(x_train[col]).reshape(-1,1))
    x_test[col]=scale.transform(np.array(x_test[col]).reshape(-1,1))

In [15]:
x_train_oe=pd.get_dummies(x_train[cat_cols])
x_test_oe=pd.get_dummies(x_test[cat_cols])

In [16]:
final_x_train_oe,final_x_test_oe=x_train_oe.align(x_test_oe,axis=1,join='inner',fill_value=0)

In [17]:
final_x_train=pd.concat([final_x_train_oe,x_train[num_cols]],axis=1)
final_x_test=pd.concat([final_x_test_oe,x_test[num_cols]],axis=1)

                         

In [18]:
print('train shape:',final_x_train_oe.shape)
print('test  shape:',final_x_test_oe.shape)

train shape: (1168, 196)
test  shape: (292, 196)


In [19]:

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score,accuracy_score,f1_score,r2_score

In [20]:
gradient=GradientBoostingRegressor()
gradient.fit(final_x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [21]:
gradient_pred=gradient.predict(final_x_test)
print(r2_score(y_test,gradient_pred))

0.9054595063447679


In [22]:
print('the train score is:', gradient.score(final_x_train,y_train))
print('the test score is:', gradient.score(final_x_test,y_test))

the train score is: 0.969775736414092
the test score is: 0.9054595063447679


In [23]:
from sklearn.model_selection import RandomizedSearchCV

In [34]:

params={'max_depth':[5,8],
        'min_samples_split':[2,4,6],
       
       'learning_rate':[.05,0.01,.1]}
rcv=RandomizedSearchCV(estimator=gradient,param_distributions=params,random_state=42,n_iter=15)
rcv.fit(final_x_train,y_train)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=15, n_jobs=1,
          param_distributions={'max_depth': [5, 8], 'min_samples_split': [2, 4, 6], 'learning_rate': [0.05, 0.01, 0.1]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [35]:
print('training score:',rcv.score(final_x_train,y_train))
print('the testing accuracy is :',rcv.score(final_x_test,y_test))

training score: 0.9853826895340779
the testing accuracy is : 0.9108659690552194


In [36]:
rcv.best_params_

{'min_samples_split': 6, 'max_depth': 5, 'learning_rate': 0.05}